# Video Processing in a Sliding Window (Experiment)

This notebook demonstrates reading a video using OpenCV and processing it **a few frames at a time**, keeping a queue of up to `batch_size` frames and processing it. 
Each time we add a new frame, if the queue is full, we drop the oldest frame. This effectively gives us a rolling/sliding window of frames.

A degenerated usage of processing 1 frame per iteration can be achieved by initializing ProcessVideoExperiment with batch_size=1.

In [ ]:
# If you need to install opencv-python, uncomment and run:
# !pip install opencv-python

import numpy as np
import cv2 as cv
from collections import deque
from datetime import datetime
from video_batch_processor import VideoBatchProcessor
from timer import Timer
from variants_extractor import DEFAULT_FARNEBACK_PARAMS, DEFAULT_LUCAS_KANADE_PARAMS, DEFAULT_TVL1_PARAMS, \
extract_farneback_variant, extract_TVL1_variant, extract_lucas_kanade_variant, extract_variants

### Set Video capture device, Batch Size, and Timer object
- `batch_size`: how many frames to store in the sliding window.
- `video_capture_object`: the object that provides frames to process. 
                          can be camera/file.
- `timer_object`: the timer object we use to make timestamps

In [ ]:
IS_WEBCAM = True  # TODO: change to False for video file analysis

if IS_WEBCAM:
        video_source = 0
else:  # video source is a video file
        video_source = "Workable Data/Processed/DPH0_Surface_TL.avi"
        
video_capture_object = cv.VideoCapture(video_source)

batch_size = 5  # Number of frames in the sliding window

if not video_capture_object.isOpened():
        if IS_WEBCAM:
                raise IOError(f"Cannot open webcam.")
        else:
                raise IOError(f"Cannot open video file: {video_source}.")

timer_object = Timer()

### Setup Hyper-Parameters
- `super_pixel_dimensions`: a tuple of (x_size, y_size)
- `frame_window_size`: the queue size of the moving window of frames.

In [ ]:
super_pixel_dimensions = (2, 2)  # can be modified
frame_window_size = 5

### Process Function
We'll define a simple function to illustrate what you might do with each batch (window) of frames.
- Here, we just print out how many frames are in the current window.
- You could replace this with custom logic (e.g., saving images, running inference, etc.).

In [ ]:
def process_frame_window(frames: deque):
    current_timestamp = datetime.now()

    if len(frames) < 2:
        raise RuntimeError("frame window size must have at least 2 frames.")
    
    frame1 = frames[0]   # take first frame in frame window
    frame2 = frames[-1]  # take last frame in frame window

    # prepare the variant extractors
    variant_extractors = dict()
    variant_extractors["Lucas-Kanade"] = {
        "kwargs": DEFAULT_LUCAS_KANADE_PARAMS,
        "function": extract_lucas_kanade_variant
    }
    variant_extractors["Farneback"] = {
        "kwargs": DEFAULT_FARNEBACK_PARAMS,
        "function": extract_farneback_variant
    }
    variant_extractors["TVL1"] = {
        "kwargs": DEFAULT_TVL1_PARAMS,
        "function": extract_TVL1_variant
    }
    # TODO: add variant extractors here

    variants = extract_variants(frame1, frame2, variant_extractors)
    
    return variants

### Setup for Main Loop

In [ ]:
frame_window = deque()

for i in range(frame_window_size):
    ret, frame = video_capture_object.read()  # Read a frame

    if not ret:
        print("End of video or error")
        break

    frame_window.append(frame)

timer_object.tick()

### Main Loop
TODO:

In [ ]:
try:
    while True:
        ret, frame = video_capture_object.read()  # Read a frame

        if not ret:
            print("End of video or error")
            break

        frame_window.popleft()
        frame_window.append(frame)

        results = process_frame_window(frame_window)

        elapsed_microseconds = timer_object.tock()

        # TODO: log the results with the elapsed microseconds

        # TODO: display updating video and graph

finally:  # release resources
    video_capture_object.release()
    cv.destroyAllWindows()


Finished processing the video.
